### 7.2.Отчет по приложению
##### Задача: соберите единый отчет по работе всего приложения. В отчете должна быть информация и по ленте новостей, и по сервису отправки сообщений. 

1. Продумайте, какие метрики необходимо отобразить в этом отчете? Как можно показать их динамику?  Приложите к отчету графики или файлы, чтобы сделать его более наглядным и информативным. Отчет должен быть не просто набором графиков или текста, а помогать отвечать бизнесу на вопросы о работе всего приложения совокупно. 
2. Автоматизируйте отправку отчета с помощью Airflow.

In [ ]:
from datetime import date, datetime, timedelta
import logging

import pandas as pd
import io
import pandahouse as ph

import telegram
import matplotlib.pyplot as plt
import seaborn as sns 

from airflow.decorators import dag, task
#from airflow.operators.python import get_current_contex


#для подключения к clickhouse
connection = {
              'host':'https://clickhouse.lab.karpov.courses',
              'database':'simulator_20221020',
              'user':'student', 
              'password':'dpo_python_2020'
             }


# Дефолтные параметры, которые прокидываются в таски
default_args = {
                'owner': 'aksakal',
                'depends_on_past': False,
                'retries': 2,
                'retry_delay': timedelta(minutes=5),
                'start_date': datetime(2022, 11, 14), # старт дата, с которой собираем данные 14/11/2022
                }

# Интервал запуска DAG
schedule_interval = '0 11 * * *' #запускаем даг каждый день в 11 часов


@dag(default_args=default_args, schedule_interval=schedule_interval, catchup=False)
def aksakal_my_bot_112022():

    # данные для подключения к боту
    my_token = 'token' # заменила настоящий токен, поскольку репозиторий публичный
    bot = telegram.Bot(token=my_token) # получаю доступ к бот
    chat_id = id_number # заменила настоящий id, поскольку репозиторий публичный



    #выгружаю данные из базы feed за последние 14 дней и сразу рассчитываю метрики ПО SOURCE за предыдущий день
    @task
    def extract_feed():
        query  = '''
                    SELECT 
                       toDate(time) as event_date,
                       dateName('weekday', toDate(time)) as week_day,
                       count(distinct user_id) as dau_feed, 
                       sum(action = 'like') as likes, 
                       sum(action = 'view') as views, 
                       round((likes /views) * 100, 2) as ctr, 
                       toISOWeek(toDate(time)) as iso_week,
                       today()- 1 as report_day,

                       (CASE
                       WHEN iso_week = toISOWeek(report_day) THEN 'this_week'
                       WHEN toISOWeek(report_day)-  iso_week = 1 THEN 'last_week'
                       WHEN toISOWeek(report_day)-  iso_week >= 2 THEN 'more then 2 week'
                       ELSE 'other week'
                       END) as status

                    FROM 
                       {db}.feed_actions 
                   WHERE 
                       toDate(time) >= today() - 14 and toDate(time) <= today()
                       and status in ('this_week', 'last_week')
                   group by 
                       event_date, week_day, status, iso_week, report_day
                  order by 
                      event_date 
                     '''
        df_cube_feed = ph.read_clickhouse(query, connection=connection)
        return df_cube_feed

    #toISOWeek(toDate(time)) as iso_week - номер недели в году
    #today()- 1 as report_day - отчетная дата, нужна для условия в CASE
    # в CASE от номера недели, на которую приходится отчетная дата, отнимаю номер недели по другим датам
    #WHERE toDate(time) >= today() - 14 and toDate(time) <= today() - беру по сегодня, иначе на графике, где только 1 точка не отображаются данные. 
    #Потом в датафрейме для текстового сообщения просто отняла данные за сегодняшний день, чтобы считать отчет за вчера


    @task
    def extract_feed_source():
        query  = '''
                    SELECT 
                       toDate(time) as event_date, 
                       dateName('weekday', toDate(time)) as week_day,
                       source,
                       count(distinct user_id) as dau_feed,
                       toISOWeek(toDate(time)) as iso_week,
                       today()- 1 as report_day,

                       (CASE
                       WHEN iso_week = toISOWeek(report_day) THEN 'this_week'
                       WHEN toISOWeek(report_day)-  iso_week = 1 THEN 'last_week'
                       WHEN toISOWeek(report_day)-  iso_week >= 2 THEN 'more then 2 week'
                       ELSE 'other week'
                       END) as status

                    FROM 
                       {db}.feed_actions 
                   WHERE 
                       toDate(time) >= today() - 14 and toDate(time) <= today()
                       and status in ('this_week', 'last_week')
                   group by 
                       event_date, week_day, source, status, iso_week, report_day
                  order by 
                      event_date 
                 '''

        df_cube_feed_source = ph.read_clickhouse(query, connection=connection)
        return df_cube_feed_source

    #sent_message - количество отправленных за день сообщений
    #dau_message - количество пользователей, отправивших за день сообщение, 
    #тех, кто получил не считаю, тк они, возможно, прочитали, но ничего не ответили - считаю, что не совершили активное действие


    @task
    def extract_message():
        query  = '''SELECT 
                        toDate(time) as event_date,
                        dateName('weekday', toDate(time)) as week_day,
                        count(distinct user_id) as dau_message,
                        count(user_id) as sent_message, 
                        toISOWeek(toDate(time)) as iso_week,
                        today()- 1 as report_day,

                        (CASE
                        WHEN iso_week = toISOWeek(report_day) THEN 'this_week'
                        WHEN toISOWeek(report_day)-  iso_week = 1 THEN 'last_week'
                        WHEN toISOWeek(report_day)-  iso_week >= 2 THEN 'more then 2 week'
                        ELSE 'other week'
                        END) as status
                   FROM 
                        {db}.message_actions 
                   WHERE  
                       toDate(time) >= today() - 14 and toDate(time) <= today()
                       and status in ('this_week', 'last_week')
                    group 
                        by event_date, week_day, status, iso_week, report_day
                    order by
                        event_date
                 '''
        df_cube_message = ph.read_clickhouse(query, connection=connection)
        return df_cube_message


    @task
    def extract_message_source():
        query  = '''SELECT 
                        toDate(time) as event_date, 
                        dateName('weekday', toDate(time)) as week_day,
                        source,
                        count(distinct user_id) as dau_message,
                        toISOWeek(toDate(time)) as iso_week,
                        today()- 1 as report_day,

                        (CASE
                        WHEN iso_week = toISOWeek(report_day) THEN 'this_week'
                        WHEN toISOWeek(report_day)-  iso_week = 1 THEN 'last_week'
                        WHEN toISOWeek(report_day)-  iso_week >= 2 THEN 'more then 2 week'
                        ELSE 'other week'
                        END) as status

                   FROM 
                        {db}.message_actions 
                   WHERE  
                       toDate(time) >= today() - 14 and toDate(time) <= today()
                       and status in ('this_week', 'last_week')
                    group by 
                        event_date, week_day, source, status, iso_week, report_day
                    order by 
                        event_date
                 '''
        df_cube_message_source = ph.read_clickhouse(query, connection=connection)
        return df_cube_message_source


    @task
    def extract_feed_stickiness(): 
        query  = ''' 
                WITH 
                    (SELECT
                        count(DISTINCT user_id) AS mau
                    FROM 
                        {db}.feed_actions
                    WHERE 
                        time >= date_add(DAY, -10, now())
                    AND 
                        time < toDate(now()) ) as t1
        
                SELECT
                      toDate(time) as event_date,
                      count(DISTINCT user_id) as dau,
                      round((dau / t1), 2) as stickiness
                FROM 
                    {db}.feed_actions
                WHERE 
                    time >= date_add(DAY, -10, now()) AND time < toDate(now())
                GROUP BY 
                    toDate(time)
            '''
        df_cube_feed_stickiness = ph.read_clickhouse(query, connection=connection)
        return df_cube_feed_stickiness

    
    @task
    def extract_message_stickiness(): 
        query  = ''' 
               WITH 
                    (SELECT
                        count(DISTINCT user_id) AS mau
                    FROM 
                        {db}.message_actions
                    WHERE 
                        time >= date_add(DAY, -10, now())
                    AND 
                        time < toDate(now()) ) as t1
        
                SELECT
                      toDate(time) as event_date,
                      count(DISTINCT user_id) as dau,
                      round((dau / t1), 2) as stickiness
                FROM 
                        {db}.message_actions
                WHERE 
                    time >= date_add(DAY, -10, now()) AND time < toDate(now())
                GROUP BY 
                    toDate(time)
                '''
        df_cube_message_stickiness = ph.read_clickhouse(query, connection=connection)
        return df_cube_message_stickiness    



#--средние значения метрик за предыдущую неделю, не включая день, за который отчет
#--беру неделю назад, тк наше приложение развивается и показатели меняются, средние за весь период меньше, чем сегодняшние
#-- тк количество активных пользователей с каждым днем по чуть-чуть растет

    @task()
    def transform_feed_week_mean(df_cube_feed):
        df_cube_feed_week_mean = df_cube_feed.query('status == "last_week"').groupby('status', as_index = False).mean().round()
        return df_cube_feed_week_mean

    @task()
    def transform_message_week_mean(df_cube_message):
        df_cube_message_week_mean = df_cube_message.query('status == "last_week"').groupby('status', as_index = False).mean().round()
        return df_cube_message_week_mean



    @task()
    def load_bot_report(df_cube_feed, df_cube_message, df_cube_feed_week_mean, \
                        df_cube_message_week_mean, df_cube_feed_stickiness, df_cube_message_stickiness):
        # получаем данные за вчера для текстового отчета
        df = df_cube_feed[df_cube_feed.event_date == max(df_cube_feed.event_date) - timedelta(days=1)]
        df1 = df_cube_message[df_cube_message.event_date == max(df_cube_message.event_date) - timedelta(days=1)]
        #отнимаю -1 день, тк  в датафрейме выводиа и сегодняшний день, иначе на графике не отображалась точка, если только данные за 1 день

    # ВАРИАНТ 1   #df= df_cube_feed.iloc[-1] 
    #df_feed['event_date'].dt.strftime("%d.%m.%Y") - для Series если с max(date)
    #df_feed['event_date'].strftime("%d.%m.%Y") - если .iloc[-1]


    # отправляем текстовое сообщение #\n - для переноса строки

        msg = "Отчет за: " + df.event_date.dt.strftime("%d.%m.%Y").values[0] +\
                "\n\nЛента новостей"+\
                "\nDAU: " + str(df.dau_feed.values[0]) + " ("+ str(df_cube_feed_week_mean.dau_feed.values[0]) +")" +\
                "\nLikes: " + str(df.likes.values[0]) + " ("+ str(df_cube_feed_week_mean.likes.values[0]) +")" +\
                "\nViews: " + str(df.views.values[0]) + " ("+ str(df_cube_feed_week_mean.views.values[0]) +")" +\
                "\nCTR: " + str(df.ctr.values[0]) + "%"+" ("+ str(df_cube_feed_week_mean.ctr.values[0]) + "%" +")" +\
                "\n\nМессенджер"+\
                "\nDAU: " + str(df1.dau_message.values[0]) + " ("+ str(df_cube_message_week_mean.dau_message.values[0]) +")"+\
                "\nSent message: " + str(df1.sent_message.values[0])+" ("+str(df_cube_message_week_mean.sent_message.values[0]) +")" +\
                "\n\n(***в скобках средние значения метрик за прошлую неделю)"            
        
    # отправляю графики

        # Пользователи по источникам трафика
        fig, axes = plt.subplots(2, 1, figsize=(10, 10))         # задаю размеры
        plt.subplots_adjust(wspace=0, hspace=0.3)
        fig.suptitle("DAU ленты и мессендежра")         # общее название графиков

        axes[0].set(title='DAU ленты')
        axes[0].set(xlabel = ' ', ylabel='DAU') 
        sns.lineplot(data=df_cube_feed, ax=axes[0], x='week_day', y='dau_feed', hue = 'status', style='status')

        axes[1].set(title='DAU мессенджера ')
        axes[1].set(xlabel = ' ', ylabel='DAU') 
        sns.lineplot(data=df_cube_message, ax=axes[1], x='week_day', y='dau_message', hue = 'status', style='status')

        # сохраняю график в буфере
        plot_object_0 = io.BytesIO()
        fig.savefig(plot_object_0)
        plot_object_0.seek(0) # перенос курсора в начало
        plot_object_0.name = 'DAU_metrics.png'
        plt.close()

        
        # Ключевые метрики за последние 7 дней
        fig, axes = plt.subplots(3, 1, figsize=(11, 20))         # задаю размеры
        plt.subplots_adjust(wspace=0, hspace=0.3)
        fig.suptitle("Ключевые метрики за последние 7 дней")         # общее название графиков

        axes[0].set(title='Likes')
        axes[0].set(xlabel = ' ', ylabel='likes') 
        sns.lineplot(data=df_cube_feed, ax=axes[0], x='week_day', y='likes', hue = 'status', style='status')

        axes[1].set(title='Views')
        axes[1].set(xlabel = ' ', ylabel='views') 
        sns.lineplot(data=df_cube_feed, ax=axes[1], x='week_day', y='views', hue = 'status', style='status')

        axes[2].set(title='Sent message')
        axes[2].set(xlabel = ' ', ylabel='message') 
        sns.lineplot(data=df_cube_message, ax=axes[2], x='week_day', y='sent_message', hue = 'status', style='status')

       # сохраняю график в буфере
        plot_object_1 = io.BytesIO()
        fig.savefig(plot_object_1)
        plot_object_1.seek(0) # перенос курсора в начало
        plot_object_1.name = 'Metrics.png'
        plt.close()

        

    #stickiness ленты и мессенджера
        fig, axes = plt.subplots(2, 1, figsize=(20, 15))         # задаю размеры
        plt.subplots_adjust(wspace=0, hspace=0.3)
        fig.suptitle("stickiness ленты и мессенджера")         # общее название графиков

        axes[0].set(title='stickiness ленты')
        axes[0].set(xlabel = ' ', ylabel='stickiness') 
        
        sns.lineplot(data=df_cube_feed_stickiness, ax=axes[0], x='event_date', y='stickiness')
        #plt.xticks(rotation=45)
        #plt.xticks получает или задает свойства расположения галочек и меток оси x.
        #rotation - это угол поворота против часовой стрелки текста этикетки оси х.

        axes[1].set(title='stickiness мессенджера')
        axes[1].set(xlabel = ' ', ylabel='stickiness') 
        sns.lineplot(data=df_cube_message_stickiness, ax=axes[1], x='event_date', y='stickiness')

    # сохраняю график в буфере
        plot_object_2 = io.BytesIO()
        fig.savefig(plot_object_2)
        plot_object_2.seek(0) # перенос курсора в начало
        plot_object_2.name = 'stickiness.png'
        plt.close()


        # отправляю сообщение и графики
        bot.sendMessage(chat_id=chat_id, text=msg)
        bot.sendPhoto(chat_id=chat_id, photo=plot_object_0)
        bot.sendPhoto(chat_id=chat_id, photo=plot_object_1)
        bot.sendPhoto(chat_id=chat_id, photo=plot_object_2)

        

    @task()
    def load_bot_picture_relplot(df_cube_feed_source, df_cube_message_source):    
    #DAU ленты/мессенджера по источнику трафика
        ax = sns.relplot(data=df_cube_feed_source, 
                kind='line',
                x='week_day',
                y='dau_feed',
                col='source',
                #row='sex', # можно еще задавать один параметр
                hue='status',
                aspect=1.10,       #соотношение сторон подграфиков определить в параметре aspect      
                ci=None,
                facet_kws={'sharex': False})
        
        ax.figure.suptitle('DAU ленты по источнику трафика')    #добавить общий заголовок к повторной диаграмме
        ax.figure.subplots_adjust(top=.8) #переместить общий заголовок немного выше, чтобы он не мешал отдельным графикам
        
    # сохраняю график в буфере
        plot_object_4 = io.BytesIO()
        ax.figure.savefig(plot_object_4)
        plot_object_4.seek(0) # перенос курсора в начало
        plot_object_4.name = 'DAU_source_1.png'
        plt.close()

        
        ax1 = sns.relplot(data=df_cube_message_source,
                kind='line',
                x='week_day',
                y='dau_message',
                col='source',
                hue='status',
                aspect=1.10, 
                ci=None,
                facet_kws={'sharex': False})
        ax1.figure.suptitle('DAU мессенджера по источнику трафика')    #добавить общий заголовок к повторной диаграмме
        ax1.figure.subplots_adjust(top=.8) #переместить общий заголовок немного выше, чтобы он не мешал отдельным графикам

    # сохраняю график в буфере
        plot_object_5 = io.BytesIO()
        ax1.figure.savefig(plot_object_5)
        plot_object_5.seek(0) # перенос курсора в начало
        plot_object_5.name = 'DAU_source_1.png'
        plt.close()


    # отправляю графики
        bot.sendPhoto(chat_id=chat_id, photo=plot_object_4)
        bot.sendPhoto(chat_id=chat_id, photo=plot_object_5)
        

        
    #запускаю task    
    
    df_cube_feed = extract_feed()
    df_cube_message = extract_message()

    df_cube_feed_source = extract_feed_source()
    df_cube_message_source = extract_message_source()

    df_cube_feed_stickiness = extract_feed_stickiness()
    df_cube_message_stickiness = extract_message_stickiness()

    df_cube_feed_week_mean = transform_feed_week_mean(df_cube_feed)
    df_cube_message_week_mean = transform_message_week_mean(df_cube_message)
    
    load_bot_report(df_cube_feed, df_cube_message, df_cube_feed_week_mean, df_cube_message_week_mean, \
                    df_cube_feed_stickiness, df_cube_message_stickiness)
    load_bot_picture_relplot(df_cube_feed_source, df_cube_message_source)
          
        
        
aksakal_my_bot_112022 = aksakal_my_bot_112022()